### This file contains following models:
#### - CNN + WE + POS
#### - LSTM
#### - Evaluation of CNN+WE, CNN+WE+POS, LSTM
#### - CNN + WE + POS + Window of size 5
#### -----------------------------------------------------------------------------------------

### Importing Dependencies and tools

In [1]:
from __future__ import print_function
import os
import numpy as np
np.random.seed(1337)

from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
import sys

Using Theano backend.
Using gpu device 0: GeForce GT 730M (CNMeM is disabled, cuDNN 5103)


### Importing Text Data

In [2]:
import xml.etree.ElementTree as ET
print('Processing text dataset')

tree = ET.parse("/home/jeet/Academics/CS671/Project/Restaurants_Train.xml")
corpus = tree.getroot()
sentences = [] # List of list of sentences.
sent = corpus.findall('.//sentence')
for s in sent:
    sentences.append(s.find('text').text)

print ('Generated list of sentences..')

MAX_SEQ_LENGTH = 69
MAX_NB_WORDS = 40000
EMBEDDING_DIM = 300

Processing text dataset
Generated list of sentences..


### Indexing Word Vectors

In [3]:
print('Indexing word vectors.')

embeddings_index = {}
f = open('glove.6B/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


#### vectorize the text samples into a 2D integer tensor and padding the sentences

In [8]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS, lower=False)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print ("Let's have a quick look at the word_index data..")

# Here padding has been done at both the ends since we will need to take the context window size of 4 units.
data = pad_sequences(sequences, maxlen=MAX_SEQ_LENGTH+2, padding='post')
print('Shape of data tensor:', data.shape)
print (data[0])
data = pad_sequences(data, maxlen=MAX_SEQ_LENGTH+4, padding='pre')
print('Shape of data tensor:', data.shape)
print (data[0])

Found 5250 unique tokens.
Let's have a quick look at the word_index data..
Shape of data tensor: (3044, 71)
[202   1  79   7  36 641   4  86   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
Shape of data tensor: (3044, 73)
[  0   0 202   1  79   7  36 641   4  86   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0]


#### defining output data

In [9]:
import nltk
from keras.preprocessing.text import text_to_word_sequence
raw_output = corpus.findall('.//sentence')
train_out= np.zeros(shape=(3044,69))
i=0
for output in raw_output:
    s = text_to_word_sequence(output.find('text').text, lower=False)
    indices = np.zeros(MAX_SEQ_LENGTH)
    
    aspectTerms = output.find('aspectTerms')
    if (aspectTerms):
        aspectTerm = aspectTerms.findall('aspectTerm')
        if (aspectTerm):
            for aspect_term in aspectTerm:
                try:
                    indices[s.index(aspect_term.attrib['term'])] = 1
#                     print (indices)
                except:
                    continue
    train_out[i] = indices
    i=i+1

print ("Shape of output tensor:", train_out.shape)

Shape of output tensor: (3044, 69)


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:11: FutureWarning: The behavior of this method will change in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.


### Preparing Embedding Layer

In [12]:
print('Preparing embedding matrix.')

# prepare embedding matrix
nb_words = len(word_index)
embedding_matrix = np.zeros((nb_words + 1, 300))
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(nb_words + 1,
                            300,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQ_LENGTH+4,
                            trainable=False)
print('Embedding Layer set..')

Preparing embedding matrix.
Embedding Layer set..


#### Extract Embeddings

In [13]:
from keras.models import Sequential
embedding_model = Sequential()
embedding_model.add(embedding_layer)

embedding_model.compile(loss='categorical_crossentropy',
                        optimizer='rmsprop',
                        metrics=['acc']
                       )
embedding_output = embedding_model.predict(data)
print('Generated word Embeddings..')
print('Shape of Embedding_output', embedding_output.shape)

Generated word Embeddings..
Shape of Embedding_output (3044, 73, 300)


#### Adding POS-tag features to input

In [16]:
from keras.preprocessing.text import text_to_word_sequence
from nltk.tag.stanford import StanfordPOSTagger
from sklearn import preprocessing
from tqdm import tqdm

train_input = np.zeros(shape=(3044,73,306))
le = preprocessing.LabelEncoder()
tags = ["CC","NN","JJ","VB","RB","IN"]
le.fit(tags)
i=0
sentences = corpus.findall('.//sentence')
for sent in sentences:
    s = text_to_word_sequence(sent.find('text').text)
    tags_for_sent = nltk.pos_tag(s)
    sent_len = len(tags_for_sent)
    ohe = [0]*6
        69
    for j in xrange(69):
        if j< len(tags_for_sent) and tags_for_sent[j][1][:2] in tags:
            ohe[le.transform(tags_for_sent[j][1][:2])] = 1
        train_input[i][j] = np.concatenate([embedding_output[i][j],ohe])
    i=i+1
    
print('Concatenated Word-Embeddings and POS Tag Features..')

Concatenated Word-Embeddings and POS Tag Features..


#### Redfining the input to incorporate the window of size 5.

In [ ]:
print (train_input.shape)
window_size = 5
final_input = np.zeros(shape=(3044,69,306,5))
for i in xrange(train_input.shape[0]):      # Access each sentence representation
    for j in xrange(train_input.shape[1]):      # Access embedded representation of each word
        if j>1 and j<71:
            for k in xrange(train_input.shape[2]):    # Access vector
#                 print (train_input[i][j-2][k],train_input[i][j-1][k],train_input[i][j][k],train_input[i][j+1][k],train_input[i][j+2][k])
                final_input[i][j-2][k] = np.array([train_input[i][j-2][k],train_input[i][j-1][k],train_input[i][j][k],train_input[i][j+1][k],train_input[i][j+2][k]],dtype=float)

print('Window features concatenated..')

(3044, 73, 306)


### WE + POS Model

In [25]:
from keras.models import Sequential
from keras.layers.convolutional import Convolution1D, Convolution2D
from keras.layers.convolutional import MaxPooling1D, MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.preprocessing import sequence
from keras.optimizers import *
from keras.regularizers import l2

In [50]:
print('Training model.')

model = Sequential()
model.add(Convolution1D(100, 5, border_mode="same", input_shape=(69, 306)))
model.add(Activation("tanh"))
model.add(MaxPooling1D(pool_length=5))
model.add(Convolution1D(50, 3, border_mode="same"))
model.add(Activation("tanh"))
model.add(MaxPooling1D(pool_length=2))
model.add(Flatten())
model.add(Dense(500))
model.add(Activation("tanh"))
# softmax classifier
model.add(Dense(69, W_regularizer=l2(0.01)))
model.add(Activation("softmax"))

model.load_weights('aspect_model_wepos.h5')
# model.compile(loss='categorical_crossentropy',
#               optimizer='rmsprop',
#               metrics=['acc'])

# print('Model Trained..')

Training model.


In [11]:
model.fit(train_input, train_out,
          validation_split=0.1,
          batch_size=10,
          nb_epoch=50
         )

Train on 2739 samples, validate on 305 samples
Epoch 1/50
2739/2739 [==============================] - 6s - loss: 2.9180 - acc: 0.2172 - val_loss: 2.2590 - val_acc: 0.4066
Epoch 2/50
2739/2739 [==============================] - 5s - loss: 2.1496 - acc: 0.3852 - val_loss: 2.0137 - val_acc: 0.5246
Epoch 3/50
2739/2739 [==============================] - 5s - loss: 1.6980 - acc: 0.4673 - val_loss: 1.8792 - val_acc: 0.4066
Epoch 4/50
2739/2739 [==============================] - 5s - loss: 1.4125 - acc: 0.5016 - val_loss: 1.8505 - val_acc: 0.3967
Epoch 5/50
2739/2739 [==============================] - 5s - loss: 1.2778 - acc: 0.5016 - val_loss: 1.7848 - val_acc: 0.4689
Epoch 6/50
2739/2739 [==============================] - 5s - loss: 1.1948 - acc: 0.5046 - val_loss: 1.6736 - val_acc: 0.4393
Epoch 7/50
2739/2739 [==============================] - 5s - loss: 1.1497 - acc: 0.5075 - val_loss: 1.7521 - val_acc: 0.3934
Epoch 8/50
2739/2739 [==============================] - 5s - loss: 1.0914 - ac

In [12]:
model.save_weights('aspect_model_wepos.h5')

### LSTM Model

In [23]:
from keras.layers.recurrent import LSTM
from keras.layers.core import TimeDistributedDense
from keras.layers import Activation

lstm_model = Sequential()
lstm_model.add(LSTM(output_dim=306,input_dim=306,return_sequences=True,activation='sigmoid',inner_activation='hard_sigmoid'))
lstm_model.add(LSTM(output_dim=306,input_dim=306,activation='sigmoid',inner_activation='hard_sigmoid'))
lstm_model.add(Activation('hard_sigmoid'))

lstm_model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [26]:
lstm_model.fit(train_input,train_out,
              validation_split=0.1,
              nb_epoch=5
              )

Train on 2739 samples, validate on 305 samples
Epoch 1/5
2739/2739 [==============================] - 47s - loss: 3.6242 - acc: 0.4830 - val_loss: 3.6045 - val_acc: 0.4721
Epoch 2/5
2739/2739 [==============================] - 47s - loss: 3.6240 - acc: 0.4830 - val_loss: 3.6048 - val_acc: 0.4721
Epoch 3/5
2739/2739 [==============================] - 45s - loss: 3.6241 - acc: 0.4830 - val_loss: 3.6044 - val_acc: 0.4721
Epoch 4/5
2739/2739 [==============================] - 46s - loss: 3.6240 - acc: 0.4794 - val_loss: 3.6053 - val_acc: 0.1607
Epoch 5/5
2739/2739 [==============================] - 46s - loss: 3.6242 - acc: 0.4794 - val_loss: 3.6045 - val_acc: 0.4721


In [27]:
lstm_model.save_weights('lstm_model.h5')

In [30]:
y_pred = model.predict(train_input[2739:])

In [34]:
print (y_pred.shape[0])

305


## Evaluating Model

### CNN+WE+POS

In [36]:
from tqdm import tqdm
processed_output = []
for i in xrange(y_pred.shape[0]):
    processed_label =[]
    for j in xrange(y_pred.shape[1]):
        if y_pred[i][j] > 0.5:
            processed_label.append(1)
        else:
            processed_label.append(0)
    processed_output.append(processed_label)

In [44]:
test_data = train_out[2739:]
total_pos = 0.0
true_pos = 0.0
total_neg = 0.0
true_neg = 0.0
for i in xrange(test_data.shape[0]):
    for j in xrange(test_data.shape[1]):
        if test_data[i][j] == 1:
            total_pos += 1
            if processed_output[i][j] ==1:
                true_pos +=1
        if test_data[i][j] == 0:
            total_neg += 1
            if processed_output[i][j] ==0:
                true_neg += 1
                

In [45]:
false_pos = total_neg-true_neg
false_neg = total_pos-true_pos

In [47]:
print ("precision %f, recall %f" % (true_pos/(true_pos+false_pos), true_pos/total_pos)))

precision 0.534562, recall 0.426471, accuracy 0.987788


### CNN+WE

In [51]:
print('Training model.')

model = Sequential()
model.add(embedding_layer)
model.add(Convolution1D(100, 5, border_mode="same", input_shape=(69, 300)))
model.add(Activation("tanh"))
model.add(MaxPooling1D(pool_length=5))
model.add(Convolution1D(50, 3, border_mode="same"))
model.add(Activation("tanh"))
model.add(MaxPooling1D(pool_length=2))
model.add(Flatten())
model.add(Dense(500))
model.add(Activation("tanh"))
# softmax classifier
model.add(Dense(69, W_regularizer=l2(0.01)))
model.add(Activation("softmax"))

model.load_weights('aspect_model.h5')

Training model.


In [52]:
y_pred = model.predict(data[2739:])

In [53]:
processed_output = []
for i in xrange(y_pred.shape[0]):
    processed_label =[]
    for j in xrange(y_pred.shape[1]):
        if y_pred[i][j] > 0.5:
            processed_label.append(1)
        else:
            processed_label.append(0)
    processed_output.append(processed_label)

In [54]:
test_data = train_out[2739:]
total_pos = 0.0
true_pos = 0.0
total_neg = 0.0
true_neg = 0.0
for i in xrange(test_data.shape[0]):
    for j in xrange(test_data.shape[1]):
        if test_data[i][j] == 1:
            total_pos += 1
            if processed_output[i][j] ==1:
                true_pos +=1
        if test_data[i][j] == 0:
            total_neg += 1
            if processed_output[i][j] ==0:
                true_neg += 1

In [ ]:
false_pos = total_neg-true_neg
false_neg = total_pos-true_pos

print ("precision %f, recall %f" % (true_pos/(true_pos+false_pos), true_pos/total_pos)))